## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
%%sh
jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [3]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [4]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [7]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3,4,5,6,7,8,9
0,0.162368,0.076959,0.204390,0.140719,-0.014264,0.094559,-0.091944,-0.010074,0.093533,-0.021663
1,0.090544,0.061001,0.118172,0.081593,-0.032196,0.031714,0.136038,0.024434,0.114184,-0.018156
2,0.108774,0.071901,0.138465,0.105755,-0.039166,0.038560,0.153961,0.025811,0.109915,-0.012207
3,-0.018281,0.087462,-0.066485,-0.050897,0.146314,0.108947,0.081757,-0.012893,-0.213363,0.208403
4,0.014779,0.110779,0.066085,-0.082850,0.103561,-0.078522,0.145314,0.046921,-0.158505,0.157855


In [8]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00213727', u'CA006030488', u'USC00214191', u'CA006020559', u'CA006034075']


,0,1,2,3,4,5,6,7,8,9
station,,,,,,,,,,
USC00213727,0.162368,0.076959,0.204390,0.140719,-0.014264,0.094559,-0.091944,-0.010074,0.093533,-0.021663
CA006030488,0.090544,0.061001,0.118172,0.081593,-0.032196,0.031714,0.136038,0.024434,0.114184,-0.018156
USC00214191,0.108774,0.071901,0.138465,0.105755,-0.039166,0.038560,0.153961,0.025811,0.109915,-0.012207
CA006020559,-0.018281,0.087462,-0.066485,-0.050897,0.146314,0.108947,0.081757,-0.012893,-0.213363,0.208403
CA006034075,0.014779,0.110779,0.066085,-0.082850,0.103561,-0.078522,0.145314,0.046921,-0.158505,0.157855


In [9]:
table=Eig.join(Stations,how='left')

In [10]:
table=table[['name','latitude','longitude','elevation',0,1,2,3]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USC00213727,HIBBING PWR SUBSTN,47.4333,-92.9667,467.0,0.162368,0.076959,0.204390,0.140719
CA006030488,BARCLAY,49.8300,-92.8200,381.0,0.090544,0.061001,0.118172,0.081593
USC00214191,KABETOGAMA,48.4089,-93.0483,365.8,0.108774,0.071901,0.138465,0.105755
CA006020559,BARWICK,48.6300,-93.9700,335.0,-0.018281,0.087462,-0.066485,-0.050897


In [11]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [12]:
(min_long, max_long, min_lat, max_lat) = (-94.8297, -92.5253, 46.3197, 49.833)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [9]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)')

NameError: name 'pdf' is not defined